In [1]:
import nltk
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('bbc-text.csv')
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


1) Data Processing

In [3]:
# a) remove digits, punctuation and making all lowercase in df
reg_for_digits = r'\d+'
reg_for_punctuation = r'[^\w\s]'
df['text'] = df['text'].replace(reg_for_digits, '',regex=True)
df['text'] = df['text'].replace(reg_for_punctuation, '',regex=True)
df['text'] = df['text'].str.lower()
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [4]:
# b) tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
df['text'] = df['text'].apply(word_tokenize)

stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

df['text'] = df['text'].apply(remove_stopwords)

In [6]:
df.head()

,category,text
0,tech,"[tv, future, hands, viewers, home, theatre, sy..."
1,business,"[worldcom, boss, left, books, alone, former, w..."
2,sport,"[tigers, wary, farrell, gamble, leicester, say..."
3,sport,"[yeading, face, newcastle, fa, cup, premiershi..."
4,entertainment,"[ocean, twelve, raids, box, office, ocean, twe..."


Implement Bag of Words

In [7]:
# let's cut our df and take first 30 rows
new_df = df.iloc[:30].copy()
new_df.head()

,category,text
0,tech,"[tv, future, hands, viewers, home, theatre, sy..."
1,business,"[worldcom, boss, left, books, alone, former, w..."
2,sport,"[tigers, wary, farrell, gamble, leicester, say..."
3,sport,"[yeading, face, newcastle, fa, cup, premiershi..."
4,entertainment,"[ocean, twelve, raids, box, office, ocean, twe..."


In [8]:
new_df.shape

(30, 2)

In [9]:
# a) create vocabulary from the training set
word_list = []
for doc in new_df['text']:
    for word in doc:
        word_list.append(word)
vocabulary = set(word_list)

In [10]:
# we create here word : index pair to navigate in the future, enumerate helps with it
word_index = {word: i for i, word in enumerate(vocabulary)}
print(word_index)

{'detention': 0, 'surface': 1, 'plasma': 2, 'decide': 3, 'unspecified': 4, 'based': 5, 'transformed': 6, 'sixyearold': 7, 'viewers': 8, 'allow': 9, 'overheads': 10, 'putting': 11, 'lose': 12, 'investor': 13, 'six': 14, 'important': 15, 'programmes': 16, 'honour': 17, 'discriminatory': 18, 'hartlepool': 19, 'vying': 20, 'plays': 21, 'weekayear': 22, 'behaving': 23, 'friday': 24, 'round': 25, 'consortium': 26, 'progress': 27, 'reflect': 28, 'shot': 29, 'co': 30, 'promised': 31, 'russell': 32, 'bbc': 33, 'may': 34, 'beheaded': 35, 'coordinator': 36, 'successfully': 37, 'political': 38, 'firm': 39, 'secured': 40, 'forecast': 41, 'exciting': 42, 'indeed': 43, 'tend': 44, 'started': 45, 'portrays': 46, 'teams': 47, 'actually': 48, 'martin': 49, 'five': 50, 'revenues': 51, 'issued': 52, 'never': 53, 'tories': 54, 'behalf': 55, 'attack': 56, 'stay': 57, 'tottenham': 58, 'roberts': 59, 'publishers': 60, 'staff': 61, 'windows': 62, 'mids': 63, 'fourthbiggest': 64, 'robredo': 65, 'regular': 66, '

In [11]:
# b) bow vector implementation
def bow_vector(tokens, word_index):
    vector = np.zeros(len(word_index))
    for token in tokens:
        if token in word_index:
            vector[word_index[token]] += 1
    return vector
# args bow vector adds second argument to function input, in pandas when we use apply, it automatically uses data as first argument, and using args (X,) we sent as tuple second argument
new_df['bow_vector'] = new_df['text'].apply(bow_vector, args=(word_index,))
new_df.head()


,category,text,bow_vector
0,tech,"[tv, future, hands, viewers, home, theatre, sy...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, ..."
1,business,"[worldcom, boss, left, books, alone, former, w...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,sport,"[tigers, wary, farrell, gamble, leicester, say...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,sport,"[yeading, face, newcastle, fa, cup, premiershi...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,entertainment,"[ocean, twelve, raids, box, office, ocean, twe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


TF-IDF

In [12]:
# implement tf
def calc_tf(bow_vector):
    total_words = bow_vector.sum()
    tf = bow_vector / total_words
    return tf
new_df['tf'] = new_df['bow_vector'].apply(calc_tf)
new_df.head()


,category,text,bow_vector,tf
0,tech,"[tv, future, hands, viewers, home, theatre, sy...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, ...","[0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0..."
1,business,"[worldcom, boss, left, books, alone, former, w...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0053763440860..."
2,sport,"[tigers, wary, farrell, gamble, leicester, say...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.007692307692307693, 0.0, 0.0..."
3,sport,"[yeading, face, newcastle, fa, cup, premiershi...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,entertainment,"[ocean, twelve, raids, box, office, ocean, twe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
def tf_vector_to_dict(tf_vector, index_map):
    """Converts a TF vector into a dictionary of {word: score}."""

    # Use a dictionary comprehension to build the result
    # It iterates through the vector, and for every score > 0,
    # it finds the word and adds it to the dictionary.
    return {
        index_map[index]: score
        for index, score in enumerate(tf_vector)
        if score > 0
    }
index_to_word = {index: word for word, index in word_index.items()}

new_df['tf_scores'] = new_df['tf'].apply(tf_vector_to_dict, args=(index_to_word,))
new_df.head()

,category,text,bow_vector,tf,tf_scores
0,tech,"[tv, future, hands, viewers, home, theatre, sy...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, ...","[0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0...","{'plasma': 0.002457002457002457, 'viewers': 0...."
1,business,"[worldcom, boss, left, books, alone, former, w...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0053763440860...","{'transformed': 0.005376344086021506, 'investo..."
2,sport,"[tigers, wary, farrell, gamble, leicester, say...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.007692307692307693, 0.0, 0.0...","{'decide': 0.007692307692307693, 'progress': 0..."
3,sport,"[yeading, face, newcastle, fa, cup, premiershi...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","{'hartlepool': 0.0038022813688212928, 'round':..."
4,entertainment,"[ocean, twelve, raids, box, office, ocean, twe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","{'five': 0.006060606060606061, 'roberts': 0.01..."


In [14]:
# idf

bow_matrix = np.stack(new_df['bow_vector'].values)
bow_matrix

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
doc_freq_arr = (bow_matrix > 0).sum(axis=0)
doc_freq = {index_to_word[i]: count for i, count in enumerate(doc_freq_arr)}
doc_freq

{'detention': np.int64(1),
 'surface': np.int64(1),
 'plasma': np.int64(1),
 'decide': np.int64(1),
 'unspecified': np.int64(1),
 'based': np.int64(1),
 'transformed': np.int64(2),
 'sixyearold': np.int64(1),
 'viewers': np.int64(1),
 'allow': np.int64(2),
 'overheads': np.int64(1),
 'putting': np.int64(2),
 'lose': np.int64(1),
 'investor': np.int64(1),
 'six': np.int64(4),
 'important': np.int64(4),
 'programmes': np.int64(1),
 'honour': np.int64(1),
 'discriminatory': np.int64(1),
 'hartlepool': np.int64(1),
 'vying': np.int64(1),
 'plays': np.int64(1),
 'weekayear': np.int64(1),
 'behaving': np.int64(1),
 'friday': np.int64(3),
 'round': np.int64(2),
 'consortium': np.int64(2),
 'progress': np.int64(2),
 'reflect': np.int64(1),
 'shot': np.int64(1),
 'co': np.int64(1),
 'promised': np.int64(1),
 'russell': np.int64(1),
 'bbc': np.int64(7),
 'may': np.int64(7),
 'beheaded': np.int64(1),
 'coordinator': np.int64(1),
 'successfully': np.int64(1),
 'political': np.int64(3),
 'firm': np

In [16]:
total_docs = len(new_df)

In [17]:
import math

In [18]:
idf = {}
for word, count in doc_freq.items():
    score = math.log((total_docs / count + 1) + 1)
    idf[word] = score

idf

{'detention': 3.4657359027997265,
 'surface': 3.4657359027997265,
 'plasma': 3.4657359027997265,
 'decide': 3.4657359027997265,
 'unspecified': 3.4657359027997265,
 'based': 3.4657359027997265,
 'transformed': 2.833213344056216,
 'sixyearold': 3.4657359027997265,
 'viewers': 3.4657359027997265,
 'allow': 2.833213344056216,
 'overheads': 3.4657359027997265,
 'putting': 2.833213344056216,
 'lose': 3.4657359027997265,
 'investor': 3.4657359027997265,
 'six': 2.2512917986064953,
 'important': 2.2512917986064953,
 'programmes': 3.4657359027997265,
 'honour': 3.4657359027997265,
 'discriminatory': 3.4657359027997265,
 'hartlepool': 3.4657359027997265,
 'vying': 3.4657359027997265,
 'plays': 3.4657359027997265,
 'weekayear': 3.4657359027997265,
 'behaving': 3.4657359027997265,
 'friday': 2.4849066497880004,
 'round': 2.833213344056216,
 'consortium': 2.833213344056216,
 'progress': 2.833213344056216,
 'reflect': 3.4657359027997265,
 'shot': 3.4657359027997265,
 'co': 3.4657359027997265,
 'pro

In [19]:
idf_val = np.array(list(idf.values()))

In [20]:
def multiply(tf):
    return np.array(tf) * idf_val
new_df['tf_idf'] = new_df['tf'].apply(multiply)

In [21]:
new_df.head()

,category,text,bow_vector,tf,tf_scores,tf_idf
0,tech,"[tv, future, hands, viewers, home, theatre, systems, plasma, highdefinition, tvs, digital, video, recorders, moving, living, room, way, people, watch, tv, radically, different, five, years, time, according, expert, panel, gathered, annual, consumer, electronics, show, las, vegas, discuss, new, technologies, impact, one, favourite, pastimes, us, leading, trend, programmes, content, delivered, viewers, via, home, networks, cable, satellite, telecoms, companies, broadband, service, providers, front, rooms, portable, devices, one, talkedabout, technologies, ces, digital, personal, video, recorders, dvr, pvr, settop, boxes, like, us, tivo, uk, sky, system, allow, people, record, store, play, pause, forward, wind, tv, programmes, want, essentially, technology, allows, much, personalised, tv, also, builtin, ...]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...]","[0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007371007371007371, 0.002457002457002457, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0, 0.002457002457002457, 0.007371007371007371, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0, 0.0, 0.007371007371007371, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002457002457002457, 0.002457002457002457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004914004914004914, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002457002457002457, 0.0, 0.002457002457002457, 0.0, 0.002457002457002457, 0.0, 0.0, 0.002457002457002457, 0.0, 0.0, 0.002457002457002457, 0.0, 0.0, ...]","{'plasma': 0.002457002457002457, 'viewers': 0.007371007371007371, 'allow': 0.002457002457002457, 'putting': 0.002457002457002457, 'important': 0.002457002457002457, 'programmes': 0.007371007371007371, 'reflect': 0.002457002457002457, 'bbc': 0.007371007371007371, 'firm': 0.002457002457002457, 'five': 0.002457002457002457, 'revenues': 0.002457002457002457, 'windows': 0.002457002457002457, 'capability': 0.004914004914004914, 'lack': 0.002457002457002457, 'viewer': 0.002457002457002457, 'set': 0.002457002457002457, 'stacey': 0.002457002457002457, 'growing': 0.002457002457002457, 'brand': 0.007371007371007371, 'sets': 0.004914004914004914, 'senior': 0.002457002457002457, 'mr': 0.004914004914004914, 'tivo': 0.007371007371007371, 'telecoms': 0.002457002457002457, 'show': 0.007371007371007371, 'alacarte': 0.002457002457002457, 'recorded': 0.002457002457002457, 'talkedabout': 0.002457002457002457, 'japan': 0.002457002457002457, 'uptake': 0.002457002457002457, 'new': 0.007371007371007371, 'group': 0.002457002457002457, 'technologies': 0.007371007371007371, 'electronics': 0.002457002457002457, 'issue': 0.002457002457002457, 'tvwatching': 0.002457002457002457, 'services': 0.002457002457002457, 'impact': 0.002457002457002457, 'network': 0.004914004914004914, 'getting': 0.002457002457002457, 'essentially': 0.002457002457002457, 'hanlon': 0.007371007371007371, 'recorders': 0.004914004914004914, 'together': 0.002457002457002457, 'website': 0.002457002457002457, 'entertainment': 0.002457002457002457, 'consumer': 0.004914004914004914, 'record': 0.002457002457002457, 'guide': 0.002457002457002457, 'portable': 0.002457002457002457, 'jolna': 0.002457002457002457, 'identity': 0.002457002457002457, 'liquid': 0.002457002457002457, 'forget': 0.002457002457002457, 'buttons': 0.002457002457002457, 'co

In [22]:
for category in new_df['category'].unique():

    category_df = new_df[new_df['category'] == category]

    category_tfidf_vectors = np.array(list(category_df['tf_idf']))

    mean_tfidf = category_tfidf_vectors.mean(axis=0)

    # Match the scores with their words
    scores = pd.Series(mean_tfidf, index=vocabulary)

    # Sort the words by score and get the top 10
    top_10 = scores.sort_values(ascending=False).head(10)

    print(top_10)


fbi           0.031507
argonaut      0.027837
mobile        0.027203
virus         0.026256
emails        0.026256
g             0.025271
people        0.023143
attachment    0.021004
technology    0.019108
phones        0.018874
dtype: float64
virgin      0.051134
blue        0.051134
ufj         0.037133
prices      0.033007
card        0.030137
worldcom    0.029813
sumitomo    0.028881
ebbers      0.026086
lg          0.025832
oil         0.025672
dtype: float64
v            0.046854
henman       0.020855
moya         0.017820
injury       0.017304
wilkinson    0.016743
cup          0.016310
farrell      0.014811
cole         0.014811
england      0.014667
club         0.014519
dtype: float64
film         0.060517
dicaprio     0.043322
scholl       0.027413
hill         0.027316
star         0.022559
wars         0.022181
ocean        0.021004
carpenter    0.020487
halloween    0.020487
sequel       0.019962
dtype: float64
hague         0.042728
mr            0.042359
howard        

In [33]:
tf_matrix = np.stack(new_df['tf'].values)
max_tf_per_word = tf_matrix.max(axis=0)

word_scores_df = pd.DataFrame({
    "word": list(vocabulary),
    "max_tf": max_tf_per_word,
    "idf": [idf[w] for w in vocabulary]
})

tf_high = word_scores_df["max_tf"].quantile(0.9)
tf_low = word_scores_df["max_tf"].quantile(0.1)
idf_high = word_scores_df["idf"].quantile(0.9)
idf_low = word_scores_df["idf"].quantile(0.1)

highTF_lowIDF = word_scores_df[
    (word_scores_df["max_tf"] >= tf_high) &
    (word_scores_df["idf"] <= idf_low)
].sort_values(by="max_tf", ascending=False)

lowTF_highIDF = word_scores_df[
    (word_scores_df["max_tf"] <= tf_low) &
    (word_scores_df["idf"] >= idf_high)
].sort_values(by="idf", ascending=False)

print("High TF & Low IDF words")
display(highTF_lowIDF.head(10))

print("Low TF & High IDF ")
display(lowTF_highIDF.head(10))


High TF & Low IDF words:


,word,max_tf,idf
137,mr,0.064516,1.609438
2587,film,0.056497,2.079442
1892,people,0.043750,1.673976
39,firm,0.043478,1.945910
821,election,0.041667,2.251292
2339,howard,0.040346,2.484907
1123,club,0.039867,2.484907
2476,technology,0.034884,2.251292
620,shares,0.032787,2.484907
1627,year,0.032787,1.553348



Low TF & High IDF words:


,word,max_tf,idf
2,plasma,0.002457,3.465736
1684,opt,0.002882,3.465736
1723,shylock,0.002882,3.465736
1720,broadcasters,0.002457,3.465736
1715,issues,0.002457,3.465736
1706,threat,0.002882,3.465736
1704,mean,0.002882,3.465736
1699,tivotogo,0.002457,3.465736
1657,ultimately,0.002457,3.465736
1768,reports,0.002882,3.465736
